# Telecom Support Graph RAG System

This notebook demonstrates how to use the Graph RAG system for telecom support.

## Setup

In [ ]:
import os
import sys
from dotenv import load_dotenv
from pathlib import Path

# Add project root to path
project_root = str(Path().absolute().parent)
if project_root not in sys.path:
    sys.path.append(project_root)

# Load environment variables
load_dotenv(os.path.join(project_root, ".env"))

## Initialize Graph Builder

In [ ]:
from src.ingestion.graph_builder import GraphBuilder

# Neo4j configuration
neo4j_config = {
    'uri': os.getenv('NEO4J_URI', 'bolt://localhost:7687'),
    'username': os.getenv('NEO4J_USER', 'neo4j'),
    'password': os.getenv('NEO4J_PASSWORD', 'your_password')
}

# Initialize GraphBuilder
graph_builder = GraphBuilder(neo4j_config)
graph_builder.connect()

## Sample Data
Let's create some sample conversation data to populate our graph.

In [ ]:
sample_conversations = [
    {
        "conversation_id": "conv1",
        "client_message": "My internet is very slow today. I can't stream any videos.",
        "agent_response": "I understand your internet is slow. Let's try restarting your router. Please unplug it for 30 seconds and plug it back in."
    },
    {
        "conversation_id": "conv2",
        "client_message": "I'm having trouble with my phone's signal. It keeps dropping calls.",
        "agent_response": "I see you're experiencing signal issues. Let's check if your phone needs a software update or if there are any network outages in your area."
    },
    {
        "conversation_id": "conv3",
        "client_message": "My bill seems higher than usual this month.",
        "agent_response": "I can help you understand your bill. Let me check your account details and explain the charges."
    }
]

## Build the Graph

In [ ]:
# Build the graph with sample data
graph_builder.build_graph(sample_conversations)
print("Graph built successfully!")


## Search the Graph

In [ ]:
def search_and_display(query, limit=3):
    print(f"\n")
    print(f"Searching for: {query}")
    results = graph_builder.search_graph_rag(query, limit=limit)
    
    print("\nMatching Conversations:")
    for i, conv in enumerate(results['conversations'], 1):
        print(f"\n{i}. Similarity: {conv['similarity']:.2f}")
        print(f"Client: {conv['client_message']}")
        print(f"Agent: {conv['agent_response']}")
    
    if results['related_entities']:
        print("\nRelated Entities:")
        for rel in results['related_entities']:
            print(f"- {rel.get('product', 'N/A')} | {rel.get('issue', 'N/A')} | {rel.get('resolution', 'N/A')}")

# Example searches
search_and_display("My internet is slow")
search_and_display("phone not working")
search_and_display("billing question")

## Clean Up

In [ ]:
# Disconnect from Neo4j when done
graph_builder.disconnect()